In [1]:
from scripts import setup_environment

setup_environment()

In [2]:
from scripts.dataset import BrainteaserDataset

dataset = BrainteaserDataset("data")

In [3]:
import string

from scripts.dataset import RiddleQuestion
from scripts.prompt_helpers import create_prompt_template


def args_generator(riddle_question: RiddleQuestion):
    template_args = {
        "question": riddle_question.question,
        "choices": "\n".join(
            [
                f"({string.ascii_uppercase[j]}) {choice}"
                for j, choice in enumerate(riddle_question.choice_list)
            ]
        ),
    }

    return template_args


chat_prompt_template = create_prompt_template("default")

In [4]:
from scripts.lmm import OllamaModelBuilder
from scripts.executor import Executor

base_url = "http://107.222.215.224:17001"
model_builder = OllamaModelBuilder(base_url)

executor = Executor(
    models=[
        # Llama3.1
        model_builder.build_model("llama3.1:8b"),
        # Llama3.2
        model_builder.build_model("llama3.2:1b"),
        model_builder.build_model("llama3.2:3b"),
        # Phi3.5
        model_builder.build_model("phi3.5:3.8b"),
        # Phi4
        model_builder.build_model("phi4:14b"),
        # Qwen2.5
        model_builder.build_model("qwen2.5:0.5b"),
        model_builder.build_model("qwen2.5:1.5b"),
        model_builder.build_model("qwen2.5:3b"),
        model_builder.build_model("qwen2.5:7b"),
        model_builder.build_model("qwen2.5:14b"),
        model_builder.build_model("qwen2.5:32b"),
        # Gemma2
        model_builder.build_model("gemma2:2b"),
        model_builder.build_model("gemma2:9b"),
        model_builder.build_model("gemma2:27b"),
        # Mistral Nemo
        model_builder.build_model("mistral-nemo:12b"),
    ]
)

2025-02-27 11:46:10,299 - INFO - Initialized executor with 15 models.


In [5]:
import numpy as np

from scripts.executor import Dataset

# Set fixed seed for reproducibility
np.random.seed(42)


def create_test_dataset(data: list[RiddleQuestion], name: str, percentage: float = 0.1):
    """Create a test dataset by randomly sampling a percentage of the original data."""
    indices = np.random.choice(
        len(data), size=int(len(data) * percentage), replace=False
    )
    return Dataset(name=name, riddles=[data[i] for i in indices])


# Create test datasets
sp_data = create_test_dataset(dataset.sp, "sp")
wp_data = create_test_dataset(dataset.wp, "wp")

# Prepare executor data
executor_data = [sp_data, wp_data]

In [6]:
from scripts.prompt_helpers import system_templates

total_results = {}

for technique in system_templates:
    chat_prompt_template = create_prompt_template(technique)
    results = await executor.aexecute(
        executor_data,
        chat_prompt_template,
        args_generator,
        dump_to_pickle=True,
        create_checkpoints=True,
        resume_from_checkpoint=True,
        run_name="zero_shot_system_prompt",
        file_name_suffix=technique,
    )

2025-02-27 11:46:10,317 - INFO - Starting execution 'zero-shot-system-prompt with suffix 'default'': 2 dataset(s) x 15 model(s) = 1665 riddle evaluations


zero-shot-system-prompt(default):   0%|          | 0/1665 [00:00<?, ?it/s]

2025-02-27 11:46:10,435 - INFO - Starting execution 'zero-shot-system-prompt with suffix 'default_improved'': 2 dataset(s) x 15 model(s) = 1665 riddle evaluations


zero-shot-system-prompt(default-improved):   0%|          | 0/1665 [00:00<?, ?it/s]

2025-02-27 11:46:10,559 - INFO - Starting execution 'zero-shot-system-prompt with suffix 'step_by_step'': 2 dataset(s) x 15 model(s) = 1665 riddle evaluations


...

2025-02-27 11:46:11,291 - INFO - Starting execution 'zero-shot-system-prompt with suffix 'pattern_matching'': 2 dataset(s) x 15 model(s) = 1665 riddle evaluations


zero-shot-system-prompt(pattern-matching):   0%|          | 0/1665 [00:00<?, ?it/s]

2025-02-27 12:04:34,237 - INFO - Saving results to results/zero-shot-system-prompt/zero-shot-system-prompt_pattern-matching_results.pkl
2025-02-27 12:04:34,681 - INFO - Starting execution 'zero-shot-system-prompt with suffix 'intuitive'': 2 dataset(s) x 15 model(s) = 1665 riddle evaluations


zero-shot-system-prompt(intuitive):   0%|          | 0/1665 [00:00<?, ?it/s]

2025-02-27 12:31:31,580 - INFO - Saving results to results/zero-shot-system-prompt/zero-shot-system-prompt_intuitive_results.pkl


## Eval results


In [2]:
import os
import glob
import pickle
from pathlib import Path

# Define the results directory path
results_dir = Path("results/zero-shot-system-prompt")

# Get all result files
result_files = glob.glob(str(results_dir / "zero-shot-system-prompt_*_results.pkl"))

# Load all results into a dictionary
# The first key is the suffix (technique name)
total_results = {}

for file_path in result_files:
    # Extract the suffix from the filename
    suffix = os.path.basename(file_path).split("_")[1]

    # Load the results from the pickle file
    with open(file_path, "rb") as f:
        wrapped_results = pickle.load(f)
        total_results[suffix] = wrapped_results.results

print(f"Loaded {len(total_results)} result sets from disk.")

Loaded 12 result sets from disk.


In [13]:
import heapq

import numpy as np
import pandas as pd
from IPython.display import display

from scripts.evaluation import eval_model_results
from scripts.prompt_helpers import system_templates


def get_best_prompt_for_each_model(input_data):
    best_prompts = {}

    # Iterate through each model
    for prompt_type, datasets in input_data.items():
        for dataset_type, models in datasets.items():
            # For each model, we need to track its best score
            for model, result in models.items():
                # Initialize the best prompt data structure for this model if not yet created
                if model not in best_prompts:
                    best_prompts[model] = {}

                # Assume eval_results returns a score based on the result data
                score = eval_model_results(result)

                # If this model doesn't have a best score for this dataset yet or if the current score is better
                if (
                    dataset_type not in best_prompts[model]
                    or score > best_prompts[model][dataset_type]["score"]
                ):
                    best_prompts[model][dataset_type] = {
                        "prompt_type": prompt_type,
                        "score": score,
                    }

    # Now best_prompts contains the best prompt type for each model and dataset
    return best_prompts


def get_best_n_prompts_for_each_model(input_data, n=3):
    best_prompts = {}

    # Iterate through each model
    for prompt_type, datasets in input_data.items():
        for dataset_type, models in datasets.items():
            for model, result in models.items():
                # Initialize the best prompt data structure for this model if not yet created
                if model not in best_prompts:
                    best_prompts[model] = {}

                # Calculate the score for the model with the current prompt type and dataset
                score = eval_model_results(result)

                # Get the prompt string length
                prompt_length = len(system_templates[prompt_type])

                # Initialize the list of prompts for this model and dataset type if not created
                if dataset_type not in best_prompts[model]:
                    best_prompts[model][dataset_type] = []

                # Append the prompt type, score, and length to the list
                best_prompts[model][dataset_type].append(
                    {
                        "prompt_type": prompt_type,
                        "prompt_length": prompt_length,
                        "score": score,
                    }
                )

    # Now sort the list of prompts for each model and dataset type and keep the top n
    top_n_prompts = {}
    for model, dataset_dict in best_prompts.items():
        top_n_prompts_for_model = {}
        for dataset_type, prompts in dataset_dict.items():
            # Get the top n prompts by sorting the list based on score (highest score first)
            sorted_prompts = heapq.nlargest(n, prompts, key=lambda x: x["score"])
            top_n_prompts_for_model[dataset_type] = sorted_prompts
        top_n_prompts[model] = top_n_prompts_for_model

    return top_n_prompts


# Get the best prompt type for each model
best_prompt_types = get_best_n_prompts_for_each_model(total_results, n=10)

# Print the results as a formatted table using pandas

for model, dataset_dict in best_prompt_types.items():
    print(f"\n{'-' * 80}\nModel: {model}")
    for dataset_type, prompts in dataset_dict.items():
        print(f"\nDataset: {dataset_type}")

        # Create a DataFrame from the prompts data
        df = pd.DataFrame(prompts)

        # Rename columns for better display
        df = df.rename(
            columns={
                "prompt_type": "Prompt Type",
                "prompt_length": "Prompt Length",
                "score": "Score",
            }
        )

        # Format the score column to 4 decimal places
        df["Score"] = df["Score"].map("{:.4f}".format)

        # Display the DataFrame
        display(df)

# Save the best prompt types
# with open("results/best_system_prompts_by_model.pkl", "wb") as f:
#     pickle.dump(best_prompt_types, f)


--------------------------------------------------------------------------------
Model: llama3.1:8b

Dataset: sp


,Prompt Type,Prompt Length,Score
0,confidence,178,59.6774
1,metaphor,187,56.4516
2,pattern-matching,176,56.4516
3,assumption-challenge,173,51.6129
4,default-improved,160,51.6129
5,elimination,182,50.0000
6,intuitive,201,48.3871
7,creative,193,46.7742
8,default,24,43.5484
9,step-by-step,251,40.3226



Dataset: wp


,Prompt Type,Prompt Length,Score
0,default-improved,160,57.1429
1,creative,193,55.1020
2,confidence,178,48.9796
3,common-sense,180,46.9388
4,default,24,46.9388
5,metaphor,187,44.8980
6,elimination,182,44.8980
7,intuitive,201,40.8163
8,perspective-shift,179,40.8163
9,pattern-matching,176,40.8163



--------------------------------------------------------------------------------
Model: llama3.2:1b

Dataset: sp


...

,Prompt Type,Prompt Length,Score
0,confidence,178,79.5918
1,elimination,182,77.5510
2,creative,193,77.5510
3,default-improved,160,77.5510
4,intuitive,201,75.5102
5,perspective-shift,179,75.5102
6,metaphor,187,75.5102
7,pattern-matching,176,75.5102
8,common-sense,180,71.4286
9,default,24,69.3878



--------------------------------------------------------------------------------
Model: mistral-nemo:12b

Dataset: sp


,Prompt Type,Prompt Length,Score
0,default-improved,160,61.2903
1,confidence,178,58.0645
2,metaphor,187,58.0645
3,creative,193,58.0645
4,intuitive,201,54.8387
5,common-sense,180,53.2258
6,step-by-step,251,51.6129
7,elimination,182,48.3871
8,pattern-matching,176,48.3871
9,perspective-shift,179,46.7742



Dataset: wp


,Prompt Type,Prompt Length,Score
0,step-by-step,251,38.7755
1,common-sense,180,30.6122
2,intuitive,201,26.5306
3,creative,193,26.5306
4,confidence,178,24.4898
5,pattern-matching,176,24.4898
6,default-improved,160,24.4898
7,default,24,24.4898
8,perspective-shift,179,22.4490
9,metaphor,187,20.4082
